### Planning 

#### Contents 

###### Classical Planning 
    - PlanningProblem
    - Action
    - Planning Problems
        - Air cargo problem
        - Spare tire problem 
        - Three block tower problem
        - Shopping problem
        - Socks and shoes problem
        - Cake problem
    - Solving Planning Problems 
        - GraphPlan
        - Linearize
        - PartialOrderPlanner

###### Planning in the Real World
    - Problem
    - HLA
    - Planning Problems
        - Job shop problem
        - Double tennis problem
    - Solving Planning Problems 
        - Hierarchical Search
        - Angelic Search

In [1]:
from planning import *
from notebook import psource

#### PlanningProblem

PDDL stands for Planning Domain Definition Language. The PlanningProblem class is used to represent planning problems in this module. We must provide the following attributes to define a problem: 
    - An initial state 
    - A set of goals 
    - A set of viable actions that can be executed in the search space of the problem

In [2]:
psource(PlanningProblem)

#### Action

To be able to model a planning problem properly, it is essential to be able to represent an action. Each action we model requires the following: 
    - preconditions that the action must meet 
    - the effects of executing the action
    - some expressions that represent the action

In [3]:
psource(Action)

Now we can set up a planning problem. Suppose we wish to plan a trip across the simplified map of Romania

In [6]:
# define simplified map
from utils import *

# define map expressions
knowledge_base = [
    expr("Connected(Bucharest,Pitesti)"),
    expr("Connected(Pitesti,Rimnicu)"),
    expr("Connected(Rimnicu,Sibiu)"),
    expr("Connected(Sibiu,Fagaras)"),
    expr("Connected(Fagaras,Bucharest)"),
    expr("Connected(Pitesti,Craiova)"),
    expr("Connected(Craiova,Rimnicu)")
    ]

# define logic propositions like symmetry and transitivity
knowledge_base.extend([
     expr("Connected(x,y) ==> Connected(y,x)"),
     expr("Connected(x,y) & Connected(y,z) ==> Connected(x,z)"),
     expr("At(Sibiu)")
    ])

# look at knowledge base
knowledge_base

[Connected(Bucharest, Pitesti),
 Connected(Pitesti, Rimnicu),
 Connected(Rimnicu, Sibiu),
 Connected(Sibiu, Fagaras),
 Connected(Fagaras, Bucharest),
 Connected(Pitesti, Craiova),
 Connected(Craiova, Rimnicu),
 (Connected(x, y) ==> Connected(y, x)),
 ((Connected(x, y) & Connected(y, z)) ==> Connected(x, z)),
 At(Sibiu)]

In [8]:
# define possible actions for our problem : Flying
#Sibiu to Bucharest
precond = 'At(Sibiu)'
effect = 'At(Bucharest) & ~At(Sibiu)'
fly_s_b = Action('Fly(Sibiu, Bucharest)', precond, effect)

#Bucharest to Sibiu
precond = 'At(Bucharest)'
effect = 'At(Sibiu) & ~At(Bucharest)'
fly_b_s = Action('Fly(Bucharest, Sibiu)', precond, effect)

#Sibiu to Craiova
precond = 'At(Sibiu)'
effect = 'At(Craiova) & ~At(Sibiu)'
fly_s_c = Action('Fly(Sibiu, Craiova)', precond, effect)

#Craiova to Sibiu
precond = 'At(Craiova)'
effect = 'At(Sibiu) & ~At(Craiova)'
fly_c_s = Action('Fly(Craiova, Sibiu)', precond, effect)

#Bucharest to Craiova
precond = 'At(Bucharest)'
effect = 'At(Craiova) & ~At(Bucharest)'
fly_b_c = Action('Fly(Bucharest, Craiova)', precond, effect)

#Craiova to Bucharest
precond = 'At(Craiova)'
effect = 'At(Bucharest) & ~At(Craiova)'
fly_c_b = Action('Fly(Craiova, Bucharest)', precond, effect)

# define driving actions
precond = 'At(x)'
effect = 'At(y) & ~At(x)'
drive = Action('Drive(x, y)', precond, effect)

In [9]:
# define goal 
goals = 'At(Bucharest)'

# define a function that will tell us when we achieve our goal
def goal_test(kb): 
    return kb.ask(expr('At(Bucharest)'))

# define the planning problem
prob = PlanningProblem(knowledge_base, goals, [fly_s_b, fly_b_s, fly_s_c, fly_c_s, fly_b_c, fly_c_b, drive])

#### Planning Problems 

##### The Air Cargo Problem

We start with cargo at two airports, SFO and JFK. Our goal is to send cargo to the other airport. We have two airplanes to help us accomplish this task. The problem has 3 actions: Load, Unload and Fly. 

In [10]:
psource(air_cargo)

In [12]:
'''
In the initial state we have cargo C1, plane P1 at airport SFO and cargo C2, plane P2 at airport JFK.
Our goal is to have cargo C1 at airport JFK and cargo C2 at airport SFO.
'''

# instantiate an air_cargo problem
airCargo = air_cargo()

# write up solution 
solution = [expr("Load(C1 , P1, SFO)"),
            expr("Fly(P1, SFO, JFK)"),
            expr("Unload(C1, P1, JFK)"),
            expr("Load(C2, P2, JFK)"),
            expr("Fly(P2, JFK, SFO)"),
            expr("Unload (C2, P2, SFO)")] 

for action in solution:
    airCargo.act(action)
    
# check if solution worked 
print(airCargo.goal_test())

True


#### The Spare Tire Problem

The problem is to change a flat tire on a car. The goal is to mount a spare tire onto the car's axle, given that we have a flat tire on the axle and a spare tire in the trunk. 

In [13]:
psource(spare_tire)

In [15]:
# instantiate 
spareTire = spare_tire()

# solve
solution = [expr("Remove(Flat, Axle)"),
            expr("Remove(Spare, Trunk)"),
            expr("PutOn(Spare, Axle)")]

for action in solution:
    spareTire.act(action)

# check if goal achieved 
print(spareTire.goal_test())

True


#### Three Block Tower Problem 

This problem's domain consists of a set of cube-shaped blocks sitting on a table. The blocks can be stacked, but only one block can fit directly on top of another. A robot arm can pick up a block and move it to another position, either on the table or on top of another block. The arm can only pick up one block at a time. The goal will be to build one or more stacks of blocks. 

In [16]:
psource(three_block_tower)

In [17]:
# instantiate
threeBlockTower = three_block_tower()

# solve
solution = [expr("MoveToTable(C, A)"),
            expr("Move(B, Table, C)"),
            expr("Move(A, Table, B)")]

for action in solution:
    threeBlockTower.act(action)

# check goal state 
print(threeBlockTower.goal_test())

True


In [18]:
# check out simpler block problem
psource(simple_blocks_world)

In [19]:
# instantiate 
simpleBlocksWorld = simple_blocks_world() 

# solve


solution = [expr('ToTable(A, B)'),
            expr('FromTable(B, A)'),
            expr('FromTable(C, B)')]

for action in solution:
    simpleBlocksWorld.act(action)

# check goal state 
print(simpleBlocksWorld.goal_test())

True


#### Shopping Problem

This problem asks us to acquire a carton of milk, a banana, and a drill. Initially we start from home and we know that the supermarket has milk and bananas and the hardware store has drills. 

In [20]:
psource(shopping_problem)

In [21]:
# instantiate 
shoppingProblem = shopping_problem() 

# solve 
solution = [expr('Go(Home, SM)'),
            expr('Buy(Milk, SM)'),
            expr('Buy(Banana, SM)'),
            expr('Go(SM, HW)'),
            expr('Buy(Drill, HW)')]

for action in solution:
    shoppingProblem.act(action)

# goal test 
shoppingProblem.goal_test()

True

#### Socks and Shoes 

Put on socks and shoes 

In [22]:
psource(socks_and_shoes)

In [23]:
# instantiate 
socksShoes = socks_and_shoes()

# solve 
solution = [expr('RightSock'),
            expr('RightShoe'),
            expr('LeftSock'),
            expr('LeftShoe')]
for action in solution:
    socksShoes.act(action)
    
# check goal state
socksShoes.goal_test()

True

#### Cake Problem 

Have a cake and eat it too. 

In [24]:
psource(have_cake_and_eat_cake_too)

In [25]:
# instantiate 
cakeProblem = have_cake_and_eat_cake_too()

# solve 
solution = [expr("Eat(Cake)"),
            expr("Bake(Cake)")]

for action in solution:
    cakeProblem.act(action)
    
# goal check
print(cakeProblem.goal_test())

True


## Planning in the Real World 

The Problem class is a wrapper for PlanningProblem with some additional functionality and data structures to handle real world planning problems that involve time and resource constraints. The Problem class includes everything that the PlanningProblem class includes. Additionally it includes a list of jobs to be done and a dictionary of resources.

It also overloads the act method to call the do_action method of the HLA class, and includes a new method refinements that finds refinements or primitive actions for high level actions. 

In [26]:
psource(Problem)

In [27]:
# High Level Action
psource(HLA)

#### Job Shop Problem 

This is a problem involving the assembly of two cars simultaneously. This consists of two jobs, each of the form [AddEngine, AddWheels, Inspect] to be performed on two cars with different requirements and availability of resources. 

In [28]:
psource(job_shop_problem)

In [29]:
# instantiate 
jobShopProblem = job_shop_problem()

# solve 
solution = [jobShopProblem.jobs[1][0],
            jobShopProblem.jobs[1][1],
            jobShopProblem.jobs[1][2],
            jobShopProblem.jobs[0][0],
            jobShopProblem.jobs[0][1],
            jobShopProblem.jobs[0][2]]

for action in solution:
    jobShopProblem.act(action)

# goal test 
print(jobShopProblem.goal_test())

True


#### Double Tennis Problem

This problem is a simple case of a multiactor planning problem, where two agents act at once and can simultaneously change the current state of the problem. A correct plan is one that, if executed by the actors, achieves the goal. In the true multiagent setting, of course, the agents may not agree to execute any particular plan, but atleast they will know what plans would work if they did agree to execute them.

In the double tennis problem, two actors A and B are playing together and can be in one of four locations: LeftBaseLine, RightBaseLine, LeftNet and RightNet. The ball can be returned only if a player is in the right place. Each action must include the actor as an argument. 

In [30]:
psource(double_tennis_problem)

In [31]:
# instantiate 
doubleTennisProblem = double_tennis_problem() 

# solve 
solution = [expr('Go(A, RightBaseLine, LeftBaseLine)'),
            expr('Hit(A, Ball, RightBaseLine)'),
            expr('Go(A, LeftNet, RightBaseLine)')]

for action in solution:
    doubleTennisProblem.act(action)

# goal test 
doubleTennisProblem.goal_test()

False